In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

data = pd.read_csv(r"data\train.csv", header=0,delimiter=",", quoting=1)
# train, val = train_test_split(data)
test = pd.read_csv(r"data\test.csv", header=0,delimiter=",", quoting=1)

In [ ]:
data.describe(include=["O"])

In [ ]:
data.info()

In [28]:
data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [29]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [ ]:
survived = data[data["Survived"]==1]
not_survived = data[data["Survived"]==0]
print(len(survived),len(not_survived),len(data))

In [ ]:
data.groupby("Pclass").Survived.mean()

In [ ]:
data[["Sex","Survived"]].groupby(["Sex"],as_index=False).mean()


In [ ]:
pd.crosstab(data["Sex"],[data["Pclass"],data["Survived"]])

In [ ]:
data[["Embarked","Survived"]].groupby("Embarked").mean()

In [ ]:
data.Embarked.value_counts()

In [ ]:
data.groupby("Pclass").Fare.describe(include="all")

In [31]:
test.isnull().sum()
# data.describe(include="all")
# data.info()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
Emb              0
FamilySize       0
FamSize          0
AgeBands         0
dtype: int64

In [ ]:
agebands = [0,1,2,3,4,5]
data["AgeBands"]=pd.cut(data["Age"],bins=[0,5,18,35,50,65,85],labels=agebands)
# data["AgeBands"]=pd.cut(data["Age"],bins=10)

In [ ]:
data[["Survived","AgeBands"]].groupby("AgeBands").mean()

In [30]:
train_test_data = [data, test]
for dataset in train_test_data:
    dataset["Sex"] = dataset["Sex"].map({"female":True, "male":False})
    dataset["Embarked"] = dataset.Embarked.fillna("S")
    dataset["Emb"] = dataset["Embarked"].map({"C":1,"Q":2,"S":3}).astype(int)
    dataset["Age"] = dataset["Age"].fillna(30)
    dataset["FamilySize"] = dataset.SibSp + dataset.Parch +1
    famlabels = [0,1,2]
    dataset["FamSize"] = pd.cut(dataset["FamilySize"],bins=[0,1,4,11],labels=famlabels)
    agebands = [0,1,2,3,4,5]
    dataset["AgeBands"]=pd.cut(dataset["Age"],bins=[0,5,18,35,50,65,85],labels=agebands)

    

In [46]:
features = ["AgeBands","Sex","Emb","Pclass","FamSize"]
data[features].isnull().sum()

AgeBands    0
Sex         0
Emb         0
Pclass      0
FamSize     0
dtype: int64

In [45]:
for feat in features:
    print(data[["Survived",feat]].groupby(feat).mean())

          Survived
AgeBands          
0         0.704545
1         0.410526
2         0.353271
3         0.398693
4         0.375000
5         0.125000
       Survived
Sex            
False  0.188908
True   0.742038
     Survived
Emb          
1    0.553571
2    0.389610
3    0.339009
        Survived
Pclass          
1       0.629630
2       0.472826
3       0.242363
         Survived
FamSize          
0        0.303538
1        0.578767
2        0.161290


In [42]:
train_x, val_x, train_y, val_y = train_test_split(data[features],data["Survived"])

In [50]:
from sklearn.neighbors import KNeighborsClassifier
clf_knn = KNeighborsClassifier().fit(train_x,train_y)
val_acc_knn = round(clf_knn.score(val_x,val_y)*100,2)
train_acc_knn = round(clf_knn.score(train_x,train_y)*100,2)
print(val_acc_knn,train_acc_knn)

76.23 84.28


c:\Users\Priya\anaconda3\lib\site-packages\sklearn\utils\validation.py:964: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  X = check_array(
c:\Users\Priya\anaconda3\lib\site-packages\sklearn\base.py:566: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  X = check_array(X, **check_params)
c:\Users\Priya\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will be

In [51]:
from sklearn.linear_model import LogisticRegression
clf_logreg = LogisticRegression().fit(train_x,train_y)
val_acc_logreg = round(clf_logreg.score(val_x,val_y)*100,2)
train_acc_logreg = round(clf_logreg.score(train_x,train_y)*100,2)
print(val_acc_logreg,train_acc_logreg)

78.92 80.39


c:\Users\Priya\anaconda3\lib\site-packages\sklearn\base.py:566: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  X = check_array(X, **check_params)
c:\Users\Priya\anaconda3\lib\site-packages\sklearn\base.py:566: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  X = check_array(X, **check_params)


In [54]:
from sklearn.svm import SVC
clf_svc = SVC().fit(train_x,train_y)
val_acc_svc = round(clf_svc.score(val_x,val_y)*100,2)
train_acc_svc = round(clf_svc.score(train_x,train_y)*100,2)
print(val_acc_svc,train_acc_svc)

80.27 82.93


In [53]:
from sklearn.tree import DecisionTreeClassifier
clf_dt = DecisionTreeClassifier().fit(train_x,train_y)
val_acc_dt = round(clf_dt.score(val_x,val_y)*100,2)
train_acc_dt = round(clf_dt.score(train_x,train_y)*100,2)
print(val_acc_dt,train_acc_dt)

75.78 85.48


In [55]:
clf_rf = RandomForestClassifier().fit(train_x,train_y)
val_acc_rf = round(clf_rf.score(val_x,val_y)*100,2)
train_acc_rf = round(clf_rf.score(train_x,train_y)*100,2)
print(val_acc_rf,train_acc_rf)

78.92 85.48


In [57]:
clf = SVC().fit(data[features],data["Survived"])
result = clf.predict(test[features])

In [60]:
output = pd.DataFrame( data={"PassengerId":test["PassengerId"], "Survived":result} )
output.to_csv("Output\\"+"RandFor.csv", index=False, quoting=3)

In [59]:
clf = RandomForestClassifier().fit(data[features],data["Survived"])
result = clf.predict(test[features])

In [ ]:
# candidate_max_leaf_nodes = [5, 25, 50, 100, 250, 500]
# candidate_estimators = [5,10,25,50,100,250]
# t_b = np.inf
# b_e = 0
# for est in candidate_estimators:
#     m_n = 0
#     m_score = np.inf
#     scores = []
#     for nodes in candidate_max_leaf_nodes:
#         forest = RandomForestClassifier(max_leaf_nodes=nodes, n_estimators=est)
#         forest.fit(x,y)
#         score = forest.score(val_x,val_y)
#         scores.append(score)
#         if score<m_score:
#             m_n = nodes
#             m_score = score
#     if m_score<t_b:
#         t_b = m_score
#         b_e = est
#     print(est," estimators, ",m_n," nodes with score: ", m_score)
#     print(scores)
# print("Best config: ",b_e)

In [ ]:
forest = RandomForestClassifier(max_leaf_nodes=5, n_estimators=5)
forest.fit(train_x,train_y)

In [ ]:
forest.fit(train_x,train_y)
pred_val = forest.predict(val_x)
pred_train = forest.predict(train_x)

In [ ]:
from sklearn.metrics import log_loss
val_error = log_loss(val_y,pred_val)
train_error = log_loss(train_y,pred_train)
print("ValError:",val_error,"\nTrainError:",train_error)

In [ ]:
result = forest.predict(test_x)

In [ ]:
output = pd.DataFrame( data={"PassengerId":test["PassengerId"], "Survived":result} )
output.to_csv("Output\\"+"RF_5_5.csv", index=False, quoting=3)